In [24]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime

import os
dfo=pd.DataFrame()

for f in os.listdir('./DataSource/HeyTea/')[-3]:
    # if any(i in f for i in ['2021-12']):
    if True:
        print(f)
        date=f.split('_')[-1].split('.')[0]
        d=pd.read_csv('./DataSource/HeyTea\%s' % f)
        dfo=pd.concat([dfo,d],axis=0)

old=pd.read_hdf(r'Processed_Data/heytea.h5',key='1227')
new=pd.concat([old,dfo])
new=new.drop_duplicates()
new.to_hdf(r'Processed_Data/heytea.h5',key='1227')

df=new.copy()
c1=df['抓取时间'].isin(['抓取时间'])
df=df[~c1]
df['datetime']=pd.to_datetime(df['抓取时间'])
df['hour']=df['datetime'].dt.hour
df['date']=df['datetime'].dt.date
df['城市']=df['城市'].str.replace('市',"")

heytea.wxapp_2020-09-23.csv
heytea.wxapp_2020-09-24.csv
heytea.wxapp_2020-09-25.csv
heytea.wxapp_2020-09-26.csv
heytea.wxapp_2021-02-08.csv
heytea.wxapp_2021-02-22.csv
heytea.wxapp_2021-02-23.csv
heytea.wxapp_2021-02-24.csv
heytea.wxapp_2021-02-25.csv
heytea.wxapp_2021-02-26.csv
heytea.wxapp_2021-02-27.csv
heytea.wxapp_2021-02-28.csv
heytea.wxapp_2021-12-01.csv
heytea.wxapp_2021-12-02.csv
heytea.wxapp_2021-12-03.csv
heytea.wxapp_2021-12-04.csv
heytea.wxapp_2021-12-05.csv
heytea.wxapp_2021-12-06.csv
heytea.wxapp_2021-12-07.csv
heytea.wxapp_2021-12-08.csv
heytea.wxapp_2021-12-09.csv
heytea.wxapp_2021-12-10.csv
heytea.wxapp_2021-12-11.csv
heytea.wxapp_2021-12-12.csv
heytea.wxapp_2021-12-13.csv
heytea.wxapp_2021-12-14.csv
heytea.wxapp_2021-12-15.csv
heytea.wxapp_2021-12-16.csv
heytea.wxapp_2021-12-17.csv
heytea.wxapp_2021-12-18.csv
heytea.wxapp_2021-12-19.csv
heytea.wxapp_2021-12-20.csv
heytea.wxapp_2021-12-21.csv
heytea.wxapp_2021-12-22.csv
heytea.wxapp_2021-12-23.csv
heytea.wxapp_2021-12

In [28]:
#选取2022年营业大于350天的店铺
c2=df['date']>=datetime.date(2021,12,20)
df=df[c2].sort_values(by=['date'])
storelist=df.groupby(by=['hour','地址'])['ID'].count().reset_index().groupby(by='地址')['ID'].mean().reset_index()
storelist=storelist[storelist['ID']>360]['地址']
df=df[df['地址'].isin(storelist)]
df['等待杯量']=pd.to_numeric(df['等待杯量'])

ht=df.groupby(by=['date','城市'])['等待杯量'].sum().reset_index()
ht['杯量_last7d_mean']=ht.groupby('城市')['等待杯量'].transform(lambda x: x.rolling(7, 7).mean())

#计算相对22/03的平均
c1=ht['date']< datetime.date(2022,3,1)
c2=ht['date']> datetime.date(2022,2,6)
Mar_mean=ht[c1&c2].groupby('城市').mean().reset_index().sort_values(by='等待杯量')[['城市','等待杯量']].reset_index(drop=True)
Mar_mean.columns=['城市', '等待杯量_mar']
ht=ht.merge(Mar_mean,on='城市')
ht['changes_since_mar']=ht['杯量_last7d_mean']/ht['等待杯量_mar']


ht['杯量_change_WoW']=(ht['杯量_last7d_mean']/ht.groupby('城市')['杯量_last7d_mean'].transform(lambda x: x.shift(periods=7)))-1

ht['date']=ht['date'].astype('string') 

def getdt(d):
    return str(d)[-5:]
ht['dt']=ht['date'].apply(getdt)

c1=ht['date'].str.contains('2022')
ht_22=ht[c1]
ht_22=ht_22[['城市', 'dt','杯量_last7d_mean']]
ht_22=ht_22.rename(columns={'杯量_last7d_mean':'杯量_22_rolling_mean'})

c2=ht['date'].str.contains('2023')
ht_23=ht[c2]
ht_23=ht_23.merge(ht_22,how='left',on=['城市','dt'])

ht_23['ht_status']=ht_23['杯量_last7d_mean']/ht_23['杯量_22_rolling_mean']


In [41]:
# covid rank vs. HEYTEA WoW

%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
# riqi='2022-12-29'
df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')

ht1=ht_23.merge(df_covid,left_on=['城市'],right_on=['城市'],how='left')

ht1['date']=ht1['date'].astype('string') 

ht2=ht1[ht1['date']==riqi].sort_values(by='杯量_last7d_mean',ascending=False)
ht2=ht2[ht1['杯量_change_WoW']<4].head(100).reset_index()

o=ht2[['城市',  'progress_2','杯量_change_WoW']]

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

# c2=o['城市'].isin(jidui_city)
# o.loc[c2,'group']='B'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi,'wow')

2023-01-03 wow


In [42]:
# ht['date']=ht['date'].astype('str')
# c1=ht['date'].isin(['2022-01-01','2022-01-02','2022-01-03'])
# c2=ht['date'].isin(['2022-12-31','2023-01-01','2023-01-02'])
# d1=ht[c1][['城市','等待杯量']].groupby('城市').mean().reset_index().rename(columns={'等待杯量':'2022元旦平均杯量'})
# d2=ht[c2][['城市','等待杯量']].groupby('城市').mean().reset_index().rename(columns={'等待杯量':'2023元旦平均杯量'})
# d3=d1.merge(d2,on='城市')
# d3['changes_since_mar']=d3['2023元旦平均杯量']/d3['2022元旦平均杯量']
# d3

In [43]:
# covid rank vs. heytea as of 2022

ht1=ht_23.merge(df_covid,left_on=['城市'],right_on=['城市'],how='left') #regular update

ht1['date']=ht1['date'].astype('string') 
ht2=ht1[ht1['date']==riqi].sort_values(by='杯量_last7d_mean',ascending=False)

ht2=ht1
ht2=ht2[ht1['ht_status']<10].head(100).reset_index()
o=ht2[['城市',  'progress_2','ht_status']]

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)

d=ht1[ht1['date']==riqi][['城市','date','ht_status']].reset_index(drop=True).rename(columns={'date':'date','ht_status':'heytea'})
d['date']=pd.to_datetime(d['date'])
d.to_hdf('Processed_Data/city_compare.h5',key='heytea')

print(riqi,'同比')


2023-01-03 同比


In [47]:
# 人均收入 vs. HEYTEA WoW

%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
# riqi='2022-12-29'


city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')

ht1=ht_23.merge(city_popu,on='城市',how='left')

ht1['date']=ht1['date'].astype('string') 

ht2=ht1[ht1['date']==riqi].sort_values(by='杯量_last7d_mean',ascending=False)
ht2=ht2[ht1['杯量_change_WoW']<4].head(100).reset_index()

o=ht2[['城市',  'GDP','杯量_change_WoW']]

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi,'wow')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2023-01-03 wow


In [46]:
# 人均收入 vs. HEYTEA as of 2022
ht2=ht1[ht1['date']==riqi].sort_values(by='杯量_last7d_mean',ascending=False)
ht2=ht2[ht1['ht_status']<10].head(100).reset_index()

o=ht2[['城市',  '人均可支配收入','ht_status']]

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi,'wow')

2023-01-03 wow


In [49]:
d1=pd.read_clipboard()
d2=pd.pivot_table(ht[ht['date'].astype('str')>'2022-12-28'],index='城市',columns='date',values='等待杯量',aggfunc='mean').reset_index()
d1.merge(d2,left_on='城市',right_on='城市',how='left').to_clipboard(index=False)